Este notebook procesa los diccionarios de variables del Censo 2024 de Chile. Lee las tablas de hogares, personas y viviendas desde un archivo Excel, limpia los valores y genera archivos de mapeo en formato JSON. Los deja listos para mapear con los microdatos.

Nota: este notebook necesita el archivo `diccionario_variables_censo2024.xlsx` descargado en el directorio `data/raw/`.

```
$ mkdir -p data/raw
$ cd data/raw
$ curl -O https://censo2024.ine.gob.cl/wp-content/uploads/2025/12/diccionario_variables_censo2024.xlsx
```

In [1]:
from pathlib import Path
import pandas as pd
import re

RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")

input_file = RAW_DIR / "diccionario_variables_censo2024.xlsx"
output_files = {
    "tabla_hogares": PROCESSED_DIR / "hogares_mapping.json",
    "tabla_personas": PROCESSED_DIR / "personas_mapping.json",
    "tabla_viviendas": PROCESSED_DIR / "viviendas_mapping.json",
}

In [2]:
def to_int_value(x):
    try:
        return pd.NA if x is None else int(x)
    except:
        return x


to_int_value(None)

<NA>

In [3]:
tabla = pd.read_excel(input_file, sheet_name="tabla_personas")
tabla.head()

,Entidad,Nombre variable,Descripción de la variable,Valor,Etiqueta de categoría,Rango,Universo,Conteo
0,Persona,id_vivienda,Llave identificadora de la vivienda o de la en...,1:7664466,Válidos,1:7664466,Todas las personas,18480432
1,Persona,id_hogar,Llave identificadora de hogar,0:26,Válidos,0:26,Todas las personas,18480432
2,Persona,id_persona,Llave identificadora de persona,0:5806,Válidos,0:5806,Todas las personas,18480432
3,Persona,region,Región,1:16,Ver diccionario códigos territoriales,1:16,Todas las personas,18480432
4,Persona,provincia,Provincia,11:163,Ver diccionario códigos territoriales,11:163,Todas las personas,18480432


In [4]:
def load_tabla(filepath, sheet_name: str) -> pd.DataFrame:
    df = pd.read_excel(
        filepath,
        index_col="Nombre variable",
        sheet_name=sheet_name,
        # thousands=",",
        # decimal=".",
    ).drop("Entidad", axis=1)

    df["Valor"] = df["Valor"].map(to_int_value)
    return df


tabla_hogares = load_tabla(input_file, sheet_name="tabla_hogares")
tabla_hogares

,Descripción de la variable,Valor,Etiqueta de categoría,Rango,Universo,Conteo
Nombre variable,,,,,,
id_vivienda,Llave identificadora de la vivienda o de la en...,1:7664466,Válidos,1:7664466,Todos los registros,6622597
id_hogar,Llave identificadora de hogar,0:26,Válidos,0:26,Todos los registros,6622597
region,Región,1:16,Ver diccionario códigos territoriales,1:16,Todos los hogares,6622597
provincia,Provincia,11:163,Ver diccionario códigos territoriales,11:163,Todos los hogares,6622597
comuna,Comuna,1101:16305,Ver diccionario códigos territoriales,1101:16305,Todos los hogares,6622597
...,...,...,...,...,...,...
tipologia_hogar,Tipología de hogar,6,Compuesto,1:7; NA (no aplica),Todos los hogares,112706
tipologia_hogar,Tipología de hogar,7,Sin núcleo,1:7; NA (no aplica),Todos los hogares,378267
tipologia_hogar,Tipología de hogar,NaN,No aplica,1:7; NA (no aplica),Todos los hogares,26070


In [5]:
def clean_str(x: str) -> str:
    if isinstance(x, str):
        # Keep all printable ASCII and extended Latin-1 (ISO-8859-1) characters (codes 32-255) and tabs/newlines
        return re.sub(r"[^\x20-\xFF\t\n]", "", x)
    return x


def create_mapping(tabla):
    mapping = {}
    for index in tabla.index:
        try:
            col_map = (
                tabla.loc[index][["Valor", "Etiqueta de categoría"]]
                .set_index("Valor")["Etiqueta de categoría"]
                .to_dict()
            )
            replace_na = lambda x: None if pd.isna(x) else x
            strip = lambda x: x.strip() if isinstance(x, str) else x
            clean = lambda x: strip(replace_na(clean_str(x)))
            col_map = {clean(k): clean(v) for k, v in col_map.items()}

            mapping[index] = col_map
        except AttributeError:
            print(f"Columna '{index}' no es un atributo mapeable.")
            pass

    return mapping


mapping = create_mapping(tabla_hogares)
mapping

Columna 'id_vivienda' no es un atributo mapeable.
Columna 'id_hogar' no es un atributo mapeable.
Columna 'region' no es un atributo mapeable.
Columna 'provincia' no es un atributo mapeable.
Columna 'comuna' no es un atributo mapeable.


{'area': {1: 'Urbano', 2: 'Rural'},
 'tipo_operativo': {1: 'Persona en Situación de Calle',
  2: 'Vivienda Particular',
  3: 'Vivienda Colectiva'},
 'p12_tenencia_viv': {1: 'Propia pagada',
  2: 'Propia pagándose',
  3: 'Arrendada con contrato',
  4: 'Arrendada sin contrato',
  5: 'Cedida por trabajo o servicio',
  6: 'Cedida por familiar u otro',
  7: 'Usufructo: solo uso y goce',
  8: 'Ocupada de hecho',
  9: 'Propiedad en sucesión o litigio',
  -99: 'No respuesta',
  None: 'No aplica'},
 'p13_comb_cocina': {1: 'Gas',
  2: 'Parafina o petróleo',
  3: 'Leña',
  4: 'Pellet',
  5: 'Carbón',
  6: 'Electricidad',
  7: 'Energía solar (ej. cocina u horno solar)',
  8: 'No utiliza fuente de energía o combustible para cocinar',
  -99: 'No respuesta',
  None: 'No aplica'},
 'p14_comb_calefaccion': {1: 'Gas',
  2: 'Parafina o petróleo',
  3: 'Leña',
  4: 'Pellet',
  5: 'Carbón',
  6: 'Electricidad',
  7: 'Otra',
  8: 'No utiliza fuente de energía o combustible para calefaccionar',
  -99: 'No re

In [6]:
import json

In [7]:
for sheet_name, output_file in output_files.items():
    print(f"Procesando {sheet_name} -> {output_file}")
    tabla = load_tabla(input_file, sheet_name=sheet_name)
    mapping = create_mapping(tabla)
    with open(output_file, "w") as f:
        json.dump(mapping, f, indent=4)

Procesando tabla_hogares -> ../data/processed/hogares_mapping.json
Columna 'id_vivienda' no es un atributo mapeable.
Columna 'id_hogar' no es un atributo mapeable.
Columna 'region' no es un atributo mapeable.
Columna 'provincia' no es un atributo mapeable.
Columna 'comuna' no es un atributo mapeable.
Procesando tabla_personas -> ../data/processed/personas_mapping.json
Columna 'id_vivienda' no es un atributo mapeable.
Columna 'id_hogar' no es un atributo mapeable.
Columna 'id_persona' no es un atributo mapeable.
Columna 'region' no es un atributo mapeable.
Columna 'provincia' no es un atributo mapeable.
Columna 'comuna' no es un atributo mapeable.
Procesando tabla_viviendas -> ../data/processed/viviendas_mapping.json
Columna 'id_vivienda' no es un atributo mapeable.
Columna 'region' no es un atributo mapeable.
Columna 'provincia' no es un atributo mapeable.
Columna 'comuna' no es un atributo mapeable.
Columna 'cant_per' no es un atributo mapeable.
Columna 'id_vivienda' no es un atributo

In [8]:
mapping["p5_num_dormitorios"]

{0: 0,
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6 o más',
 -99: 'No respuesta',
 None: 'No aplica'}

In [9]:
viviendas_test = pd.read_parquet(
    "../data/processed/viviendas_censo2024.parquet", filters=[[("comuna", "==", 5802)]]
)
viviendas_test.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/viviendas_censo2024.parquet'

In [ ]:
def map_columns(df: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    def get_key(x):
        return col_map.get(x, x)

    for col, col_map in mapping.items():
        if col in df.columns:
            df[col] = df[col].map(get_key)
    return df


viviendas_test_mapped = map_columns(viviendas_test.copy(), mapping)
viviendas_test_mapped.head()

,id_vivienda,region,provincia,comuna,comuna_bajo_umbral,area,tipo_operativo,cant_hog,cant_per,p2_tipo_vivienda,...,p5_num_dormitorios,p6_fuente_agua,p7_distrib_agua,p8_serv_hig,p9_fuente_elect,p10_basura,p11a_num_personas,p11b_comparte_gasto,p11c_num_hogar,indice_hacinamiento
0,1,5,58,5802,No,Urbano,Vivienda particular,1.0,4,Casa,...,2,Red pública,Con llave dentro de la vivienda,"Dentro de la vivienda, conectado a una red de ...",Red pública,La recogen los servicios de aseo,4.0,Sí,NaN,Vivienda sin hacinamiento
1,598,5,58,5802,No,Urbano,Vivienda particular,1.0,1,Casa,...,1,Red pública,Con llave dentro de la vivienda,"Dentro de la vivienda, conectado a una red de ...",Red pública,La recogen los servicios de aseo,1.0,NaN,NaN,Vivienda sin hacinamiento
2,1100,5,58,5802,No,Urbano,Vivienda particular,1.0,2,Departamento,...,2,Red pública,Con llave dentro de la vivienda,"Dentro de la vivienda, conectado a una red de ...",Red pública,La recogen los servicios de aseo,2.0,Sí,NaN,Vivienda sin hacinamiento
3,1346,5,58,5802,No,Urbano,Vivienda particular,1.0,4,Casa,...,3,Red pública,Con llave dentro de la vivienda,"Dentro de la vivienda, conectado a una red de ...",Red pública,La recogen los servicios de aseo,4.0,Sí,NaN,Vivienda sin hacinamiento
4,1847,5,58,5802,No,Urbano,Vivienda particular,1.0,4,Casa,...,1,Red pública,Con llave dentro de la vivienda,"Dentro de la vivienda, conectado a una red de ...",Red pública,La recogen los servicios de aseo,4.0,Sí,NaN,Vivienda con hacinamiento medio


In [ ]:
viviendas_test_mapped.p11c_num_hogar.value_counts()

p11c_num_hogar
2.0     274
3.0      32
4.0       7
8.0       1
10.0      1
Name: count, dtype: int64